In [1]:
import selenium
import re
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
from cmath import nan
from geopy.geocoders import Nominatim

def initialize():
    #CHROMEDRIVER_PATH = '/opt/WebDriver/bin'
    WINDOW_SIZE = "1000,1024"


    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    #chrome_options.binary_location = CHROME_PATH

    # Creating a chromedriver instance
    driver = webdriver.Chrome('D:\DATA\DV\chromedriver.exe')#executable_path=CHROMEDRIVER_PATH, 

    driver.get("https://www.boligsiden.dk/postnummer/2700/tilsalg")
    driver.implicitly_wait(2)#It instructs the webdriver to wait for a certain amount of time before elements load on the webpage.

    driver.find_element(By.XPATH, "//button[contains(text(), 'OK')]").click()

    return driver

driver = initialize()

def zip_cph():
    zip_df = pd.read_csv('zip_cph.csv', header=None)

    postal_areas = []

    for i in zip_df[1]:
        numbers = i.replace("'", "")
        numbers = numbers.replace(",", "")
        numbers = numbers.split()
        for number in numbers:
            number = number.replace("[", "")
            number = number.replace("]", "")
            postal_areas.append(number)
    return postal_areas

def zip_cph_extended():
    zip_df = pd.read_csv('zip_cph_extended.csv', header=None)

    postal_areas = []

    for i in zip_df[1]:
        numbers = i.replace("'", "")
        numbers = numbers.replace(",", "")
        numbers = numbers.split()
        for number in numbers:
            number = number.replace("[", "")
            number = number.replace("]", "")
            postal_areas.append(number)
    return postal_areas


def searchzip(postal_areas):
    searchzip = []
    for area in postal_areas:
        a = ("https://www.boligsiden.dk/postnummer/" + area +"/tilsalg")
        print(a)
        searchzip.append(a)
    return searchzip

def get_houselinks(searchzip):
    proceed = True
    houselinks = []

    for search in searchzip:
        time.sleep(1)
        driver.get(search)
        proceed = True

        while proceed:
            time.sleep(1)
            houses = driver.find_elements(By.XPATH, "//a[contains(@href, 'adresse') and contains(text(), 'Se bolig')]")
            for house in houses:
                try:
                    houselink = house.get_attribute("href")
                    houselinks.append(houselink)
                except:
                    pass
            try:
                nextbutton = driver.find_element(By.XPATH, "//a[@aria-label= 'Next page' and @aria-disabled = 'false']")
                nextbutton.click()
            except:
                proceed = False
    return houselinks

def create_dataframe():
    df = pd.DataFrame(columns = ['Type', 'Address', 'Price', 'Sqm Price', 'Size', 'Living Area', 'Ground Area', 'Rooms', 'Ownership Expense', 'Year Built', 'Story', 'Energy Label', 'Description'])
    return df

def scrape(houselinks, df):
    for link in houselinks:
        try:
            driver.get(link)
            driver.implicitly_wait(2)

            try:
                house_address = driver.find_element(By.XPATH, '//h1').text
            except: 
                house_address = "nan"

            try:
                type = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[2]/div/div[4]/div/div/p').text
            except: 
                type = "nan"

            try:
                house_price = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[3]/div[1]/div[1]/div[1]/h2').text
            except: 
                house_price = "nan" 

            try:   
                price_sqm = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[3]/div[1]/div[3]/div[1]/div/div/div/div/span').text
            except: 
                price_sqm = "nan"

            try:
                house_size = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[3]/div[2]/div/div[1]/div/div[2]/div/div/span').text
            except: 
                house_size = "nan"

            try:
                living_area = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[4]/div[6]/div[2]').text
            except: 
                living_area = "nan"

            try:
                ground_area = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[3]/div[2]/div/div[2]/div/div[2]/div/div/span').text
            except: 
                ground_area = "nan"

            try:
                rooms = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[3]/div[2]/div/div[3]/div/span').text
            except: 
                rooms = "nan"

            try:
                ownership_expense = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[3]/div[2]/div/div[4]/div/div[2]/div/div/span').text
            except: 
                ownership_expense = "nan"

            try:
                see_more_button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[3]/div[2]/div/button/span')
                driver.execute_script('arguments[0].click()', see_more_button)
            except:
                pass

            try:
                year_built = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[3]/div[2]/div/div[5]/div/span').text
            except: 
                year_built = "nan"

            try:
                story = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[4]/div[2]/div[2]/font/font').text
            except:
                story = "st"

            try:
                energy_label = driver.find_element(By.XPATH, '//*[@id="Lag_1"]/title').text
            except:
                energy_label = "nan"  

            try:
                description = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/main/div/div[3]/div/div[1]/div/div[6]/div/div[1]/div/p').text
            except:
                description = "nan"  

            df = df.append({'Type': type, 'Address' : house_address, 'Price' : house_price, 'Sqm Price' : price_sqm, 'Size': house_size, 'Living Area' : living_area, 'Ground Area': ground_area, 'Rooms': rooms, 'Ownership Expense': ownership_expense, 'Year Built': year_built, 'Story': story, 'Energy Label': energy_label, 'Description': description}, 
                        ignore_index = True)

        except:
            pass
        
    return df

def get_coordinates(df):
    geolocator = Nominatim(user_agent="my_user_agent")
    addresses = df['Address'].values
    country ="Dk"
    longitude = []
    latitude=[]
    cleaned_address = []

    for address in addresses:
        address = re.sub("(?i)\d+\.+\s(th.|tv.|st.|mf.|th,|tv,|st,|mf,|th|tv|st|mf)", "", address)
        address = re.sub("\d+\s(th.|tv.|st.|mf.|th,|tv,|st,|mf,|th|tv|st|mf)", "", address)
        address = re.sub("\s(th.|tv.|st.|mf.|th,|tv,|st,|mf,|th|tv|st|mf)", "", address)
        address = address.replace(",", "")
        address = address.replace(".", "")
        print(address)
        cleaned_address.append(address)
        try:
            loc = geolocator.geocode(address)
            if loc:
                print(address)
                print(loc)
                print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)
                longitude.append(loc.longitude)
                latitude.append(loc.latitude)
            else:
                print(address)
                print("not found")
                longitude.append("nan")
                latitude.append("nan")
        except:
            print(address)
            print("not found")
            longitude.append("nan")
            latitude.append("nan")

    df["Longitude"] = longitude
    df["Latitude"] = latitude
    df["Cleaned_address"] = cleaned_address
    return df

def save_df(df, name):
    df.to_csv(name+'.csv') 

def append_df(df, df2):
   data = df.append(df2, ignore_index = True)
   return data  


def run_script():

    zips = searchzip(zip_cph())

    #zips = searchzip(zip_cph_extended()) uncomment if the scraper should be ran for the outskirts

    houselinks = get_houselinks(zips)

    df = create_dataframe()

    scraped_df = scrape(houselinks, df)

    df = get_coordinates(scraped_df)

    save_df(df, "scraped_data_boligsiden") # provide the name on how to save the .csv

    return df


run_script()


C:\Users\kovac\AppData\Local\Temp\ipykernel_32884\712958447.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('D:\DATA\DV\chromedriver.exe')#executable_path=CHROMEDRIVER_PATH,


https://www.boligsiden.dk/postnummer/2700/tilsalg
https://www.boligsiden.dk/postnummer/1050/tilsalg
https://www.boligsiden.dk/postnummer/1051/tilsalg
https://www.boligsiden.dk/postnummer/1052/tilsalg
https://www.boligsiden.dk/postnummer/1053/tilsalg
https://www.boligsiden.dk/postnummer/1054/tilsalg
https://www.boligsiden.dk/postnummer/1055/tilsalg
https://www.boligsiden.dk/postnummer/1056/tilsalg
https://www.boligsiden.dk/postnummer/1057/tilsalg
https://www.boligsiden.dk/postnummer/1058/tilsalg
https://www.boligsiden.dk/postnummer/1059/tilsalg
https://www.boligsiden.dk/postnummer/1060/tilsalg
https://www.boligsiden.dk/postnummer/1061/tilsalg
https://www.boligsiden.dk/postnummer/1062/tilsalg
https://www.boligsiden.dk/postnummer/1063/tilsalg
https://www.boligsiden.dk/postnummer/1064/tilsalg
https://www.boligsiden.dk/postnummer/1065/tilsalg
https://www.boligsiden.dk/postnummer/1066/tilsalg
https://www.boligsiden.dk/postnummer/1067/tilsalg
https://www.boligsiden.dk/postnummer/1068/tilsalg


C:\Users\kovac\AppData\Local\Temp\ipykernel_32884\712958447.py:184: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Type': type, 'Address' : house_address, 'Price' : house_price, 'Sqm Price' : price_sqm, 'Size': house_size, 'Living Area' : living_area, 'Ground Area': ground_area, 'Rooms': rooms, 'Ownership Expense': ownership_expense, 'Year Built': year_built, 'Story': story, 'Energy Label': energy_label, 'Description': description},
C:\Users\kovac\AppData\Local\Temp\ipykernel_32884\712958447.py:184: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Type': type, 'Address' : house_address, 'Price' : house_price, 'Sqm Price' : price_sqm, 'Size': house_size, 'Living Area' : living_area, 'Ground Area': ground_area, 'Rooms': rooms, 'Ownership Expense': ownership_expense, 'Year Built': yea